In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/thesis/covid_networks

Mounted at /content/gdrive
/content/gdrive/My Drive/thesis/covid_networks


## setup and Loading

In [32]:
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import math
import networkx as nx
from sklearn.linear_model import LinearRegression
import sklearn as sc
import plotly.graph_objects as go
import utils as util
import random
import joblib
import statsmodels.api as sm
import plotly.express as px
import os
from collections import defaultdict

%matplotlib inline
plt.style.use('seaborn-white')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import utils as util
W, num_genes, gene_indexes, gene_scores = util.load_network_scores()
random_networks = util.load_random_networks()
interactions = util.load_and_map_interactions()
#Vascular Permeability
perm3 = pd.read_csv("covid_files/data/inputs/perm3.csv")  # permeability after 3 Days
perm4 = pd.read_csv("covid_files/data/inputs/perm4.csv")  # permeability after 4 Days
# all target protiens
all_targets = util.load_all_targets()

querying 1-332...done.
Finished.


In [57]:
# grouping count
interaction_count = interactions.groupby("viral", as_index=False).count()
interaction_count["viral"] = interaction_count.apply(lambda row: row["viral"].lower(), axis=1)

# %%
# lower case the protiens names
perm3["Viral Protein SARS-Cov-2"] = perm3.apply(lambda row: row["Viral Protein SARS-Cov-2"].lower(), axis=1)
perm4["Viral Protein SARS-Cov-2"] = perm4.apply(lambda row: row["Viral Protein SARS-Cov-2"].lower(), axis=1)

# join permeabilty and interactions
corr_data = interaction_count.merge(perm3, left_on="viral", right_on="Viral Protein SARS-Cov-2")
corr_data = corr_data.merge(perm4, left_on="viral", right_on="Viral Protein SARS-Cov-2")

ongoing_data = pd.DataFrame(columns=['viral', 'interactions_count', 'perm_3_days'])
ongoing_data['viral'] = corr_data['viral']
ongoing_data['interactions_count'] = corr_data['Preys']
ongoing_data['perm_3_days'] = corr_data['Permeability Value (3 days)']
ongoing_data['perm_4_days'] = corr_data['Permeability Value (4 days)']
del(corr_data)

## 1. Perm Correlation

### random sources

In [84]:
# calculate p-value and scores by random sources
viral_scores = []
viral_p_values = []
temp_data = ongoing_data.copy()
for index, row in ongoing_data.iterrows():
    runs = []
    # calculate the score of viral gene
    sources = interactions[interactions["viral"].str.lower() == row["viral"]]["entrezgene"].to_list()
    score = sum([sum(v[target_index])for k, v in gene_scores.items() if k in sources])
    runs.append(score)
    viral_scores.append(score)

    # generate 100 random scores
    for i in range(100):
        random_sources = random.sample(population=list(g.nodes), k=len(sources))
        score = sum([sum(v[target_index]) for k, v in gene_scores.items() if k in random_sources])
        runs.append(score)

    pvalue = (102 - sp.stats.rankdata(runs, method="ordinal")[0]) / 101
    viral_p_values.append(pvalue)

temp_data["network_score"] = viral_scores
temp_data["pvalue"] = viral_p_values
temp_data["adjusted_pvalue"] = sm.stats.multipletests(temp_data["pvalue"], alpha=0.05, method="fdr_bh", is_sorted=False)[1]
temp_data.to_csv('covid_files/1_perm_correlation/1.1_data.csv',index=False)

#### 1.1 

In [104]:
data = pd.read_csv('covid_files/1_perm_correlation/1.1_data.csv')
# correlation of number of adj-pvalue random sources
correlation_4 = round(sp.stats.pearsonr(data["adjusted_pvalue"], data["perm_3_days"])[0], 4)
fig = px.scatter(data, x="adjusted_pvalue", y="perm_3_days", text="viral")
fig.update_traces(textposition="top center")
fig.update_layout(height=800, title_text=f"""Adj(BH) P-value(random sources) Vs Permeability, Pearson={correlation_4}""")
fig.write_html('covid_files/1_perm_correlation/1.1.html')
fig.show()

### Random Nieghbors with same degree

In [88]:
temp_data = ongoing_data.copy()
# random neighbours with same degree
genes_by_degree = defaultdict(list)
degrees_dict = dict(g.degree)
for key, val in sorted(degrees_dict.items()):
    genes_by_degree[val].append(key)

viral_scores = []
viral_p_values = []
for index, row in ongoing_data.iterrows():
    runs = []
    # calculate the score of viral gene
    sources = interactions[interactions["viral"].str.lower() == row["viral"]]["entrezgene"].to_list()
    score = sum([sum(v[target_index]) for k, v in gene_scores.items() if k in sources]) / len(sources)
    runs.append(score)
    viral_scores.append(score)

    # generate 100 random scores
    for i in range(100):
        random_sources = []
        for s in sources:
            random_sources.append(random.sample(population=genes_by_degree[g.degree(s)], k=1)[0])
        score = sum([sum(v[target_index]) for k, v in gene_scores.items() if k in random_sources]) / len(sources)
        runs.append(score)

    pvalue = (102 - sp.stats.rankdata(runs, method="ordinal")[0]) / 101
    viral_p_values.append(pvalue)

temp_data["pvalue_random_source_same_degree"] = viral_p_values
temp_data["adjusted_pvalue_random_source_same_degree"] = sm.stats.multipletests(
    temp_data["pvalue_random_source_same_degree"], alpha=0.05, method="fdr_bh", is_sorted=False)[1]
temp_data.to_csv('covid_files/1_perm_correlation/1.2_data.csv',index=False)

#### 1.2

In [96]:
data = pd.read_csv('covid_files/1_perm_correlation/1.2_data.csv')
# correlation of number of adj-pvalue random sources same degree
correlation_6 = round(sp.stats.pearsonr(data["adjusted_pvalue_random_source_same_degree"], data["perm_3_days"])[0], 4,)
fig = px.scatter(data, x="adjusted_pvalue_random_source_same_degree", y="perm_3_days", text="viral")
fig.update_traces(textposition="top center")
fig.update_layout(height=800, title_text=f"""P-value(random source same degree) Vs Permeability, Pearson={correlation_6}""")
fig.write_html('covid_files/1_perm_correlation/1.2.html')
fig.show()

### Random Networks

In [ ]:
temp_data = ongoing_data.copy()
# calculate p-value and scores multi-Nets
viral_p_values = []
for index, row in ongoing_data.iterrows():
    runs = []
    # calculate the score of viral gene
    sources = interactions[interactions["viral"].str.lower() == row["viral"]]["entrezgene"].to_list()
    score = sum([sum(v[target_index]) for k, v in gene_scores.items() if k in sources])
    runs.append(score)

    # generate 100 random scores by 100 random networks
    for net, prop in random_networks.items():
        score = sum([sum(v[target_index]) for k, v in prop.items() if k in sources])
        runs.append(score)

    pvalue = (102 - sp.stats.rankdata(runs, method="ordinal")[0]) / 101
    viral_p_values.append(pvalue)

temp_data["pvalue_multi_net"] = viral_p_values
temp_data["adjusted_pvalue_multi_net"] = sm.stats.multipletests(ongoing_data["pvalue_multi_net"], alpha=0.05, method="fdr_bh", is_sorted=False)[1]
temp_data.to_csv('covid_files/1_perm_correlation/1.3_data.csv',index=False)

#### 1.3

In [ ]:
data = pd.read_csv('covid_files/1_perm_correlation/1.3_data.csv')
# correlation of number of adj-pvalue random networks
correlation_5 = round(sp.stats.pearsonr(data["adjusted_pvalue_multi_net"], data["perm_3_days"])[0], 4)
fig = px.scatter(data, x="adjusted_pvalue_multi_net", y="perm_3_days", text="viral")
fig.update_traces(textposition="top center")
fig.update_layout(height=800, title_text=f"""Adjusted P-value(multi-net) Vs Permeability, Pearson={correlation_5}""")
fig.write_html('covid_files/1_perm_correlation/1.3.html')
fig.show()

# unsorted

In [59]:
def calc_pvalue(data, by_vascular = True):
  p_values = pd.DataFrame()
  scores = pd.DataFrame()    
  if by_vascular:
    for tar_name, matrix in data.items():
      p_values[tar_name + "_pvalue"] = ((101 - sp.stats.rankdata(matrix, method="ordinal",axis=1)) / 101)[:,0]
      scores[tar_name + "_score"] = matrix[:,0].A1
  else:
    sum_matrix = sum(data.values())
    p_values["pvalue_of_sum"] = ((101 - sp.stats.rankdata(sum_matrix, method="ordinal",axis=1)) / 101)[:,0]
    scores["score_of_sum"] = sum_matrix[:,0].A1
  return p_values, scores

In [ ]:
p_values, scores = calc_pvalue(random_by_interactions)
Y = ongoing_data[['perm_3_days','perm_4_days']]

In [ ]:
#random nieghbours 
p_values, scores = calc_pvalue(random_by_interactions)
Y = ongoing_data[['perm_3_days','perm_4_days']]
cca = CCA(n_components=1)
p_values_r, Y_r = cca.fit_transform(p_values, Y) 
print('P-value random neighbours: \n',np.corrcoef(p_values_r.T, Y_r.T))

P-value random neighbours: 
 [[1.         0.75393226]
 [0.75393226 1.        ]]


In [ ]:
#scores
cca = CCA(n_components=1)
scores_r, Y_r = cca.fit_transform(scores, Y) 
print('absolute scores: \n',np.corrcoef(scores_r.T, Y_r.T))

absolute scores: 
 [[1.         0.77283989]
 [0.77283989 1.        ]]


In [ ]:
# random networks
p_values, scores = calc_pvalue(random_by_netwroks)
# Y = ongoing_data[['perm_3_days','perm_4_days']]
Y = ongoing_data[['perm_3_days']]
cca = CCA(n_components=1)
p_values_r, Y_r = cca.fit_transform(np.log(p_values+0.00001)*(-1), Y) 
print('P-value random networks: \n',np.corrcoef(p_values_r.T, Y_r.T))

P-value random networks: 
 [[1.         0.66093226]
 [0.66093226 1.        ]]


In [ ]:
#average score
cca = CCA(n_components=1)
scores_r, Y_r = cca.fit_transform(scores.div(ongoing_data.interactions_count,axis=0), Y) 
print('average score: \n',np.corrcoef(scores_r.T, Y_r.T))

average score: 
 [[1.         0.86312274]
 [0.86312274 1.        ]]


In [ ]:
{name:round(w,4) for name,w in zip(list(scores.columns),list(cca.x_weights_[:,0]))}

{'Cadherin-2_score': 0.1511,
 'Cadherin-3_score': -0.1354,
 'Cadherin-4_score': -0.1262,
 'Cadherin-5_score': -0.4801,
 'Catenin delta 1_score': 0.4059,
 'Cateninβ  _score': 0.5876,
 'Claudin-5_score': 0.2187,
 'Occludin_score': -0.1224,
 'ZO-1_score': 0.0293,
 'ZO-2_score': -0.2779,
 'ZO-3_score': 0.0422,
 'α Catenin _score': -0.2444}

In [ ]:
{name:round(w,4) for name,w in zip(list(Y.columns),list(cca.y_weights_[:,0]))}

{'perm_3_days': 0.99, 'perm_4_days': -0.141}

In [ ]:
correlations = []
for i in range(101):
  temp_scores = pd.DataFrame()
  for tar_name, matrix in random_by_netwroks.items():
    temp_scores[tar_name] = matrix[:,i].A1
  cca = CCA(n_components=1)
  Y = ongoing_data[['perm_3_days','perm_4_days']]
  scores_r, Y_r = cca.fit_transform(temp_scores.div(ongoing_data.interactions_count,axis=0), Y) 
  correlations.append(np.corrcoef(scores_r.T, Y_r.T)[0,1])


In [ ]:
((101 - sp.stats.rankdata(correlations, method="ordinal")) / 101)[0]

0.039603960396039604

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=True, with_std=True)
# X_sc = sc.fit_transform(scores)
# Y_sc = sc.fit_transform(p_values)

In [21]:
# %%
# calculate p-value and scores by random sources
viral_scores = []
viral_p_values = []
for index, row in ongoing_data.iterrows():
    runs = []
    # calculate the score of viral gene
    sources = interactions[interactions["viral"].str.lower() == row["viral"]]["entrezgene"].to_list()
    score = sum([sum(v[target_index])for k, v in gene_scores.items() if k in sources])
    runs.append(score)
    viral_scores.append(score)

    # generate 100 random scores
    for i in range(100):
        random_sources = random.sample(population=list(g.nodes), k=len(sources))
        score = sum([sum(v[target_index]) for k, v in gene_scores.items() if k in random_sources])
        runs.append(score)

    pvalue = (102 - sp.stats.rankdata(runs, method="ordinal")[0]) / 101
    viral_p_values.append(pvalue)

ongoing_data["network_score"] = viral_scores
ongoing_data["pvalue"] = viral_p_values
ongoing_data["adjusted_pvalue"] = sm.stats.multipletests(ongoing_data["pvalue"], alpha=0.05, method="fdr_bh", is_sorted=False)[1]


In [22]:
# %%
# random neighbours with same degree
genes_by_degree = defaultdict(list)
degrees_dict = dict(g.degree)
for key, val in sorted(degrees_dict.items()):
    genes_by_degree[val].append(key)

viral_scores = []
viral_p_values = []
for index, row in ongoing_data.iterrows():
    runs = []
    # calculate the score of viral gene
    sources = interactions[interactions["viral"].str.lower() == row["viral"]]["entrezgene"].to_list()
    score = sum([sum(v[target_index]) for k, v in gene_scores.items() if k in sources]) / len(sources)
    runs.append(score)
    viral_scores.append(score)

    # generate 100 random scores
    for i in range(100):
        random_sources = []
        for s in sources:
            random_sources.append(random.sample(population=genes_by_degree[g.degree(s)], k=1)[0])
        score = sum([sum(v[target_index]) for k, v in gene_scores.items() if k in random_sources]) / len(sources)
        runs.append(score)

    pvalue = (102 - sp.stats.rankdata(runs, method="ordinal")[0]) / 101
    viral_p_values.append(pvalue)

ongoing_data["pvalue_random_source_same_degree"] = viral_p_values
ongoing_data["adjusted_pvalue_random_source_same_degree"] = sm.stats.multipletests(
    ongoing_data["pvalue_random_source_same_degree"], alpha=0.05, method="fdr_bh", is_sorted=False)[1]

# %%
# calculate score by target
for tar_id, name in zip(target, target_names):
    target_score = []
    viral_p_values = []
    for index, row in ongoing_data.iterrows():
        runs = []
        # calculate the score of viral gene
        sources = interactions[interactions["viral"].str.lower() == row["viral"]]["entrezgene"].to_list()
        score = sum([v[gene_indexes[tar_id]] for k, v in gene_scores.items() if k in sources]) / len(sources)
        runs.append(score)
        target_score.append(score)
        # generate 100 random scores
        for i in range(100):
            random_sources = random.sample(population=list(g.nodes), k=len(sources))
            score = sum([v[gene_indexes[tar_id]] for k, v in gene_scores.items() if k in random_sources]) / len(sources)
            runs.append(score)
        pvalue = (102 - sp.stats.rankdata(runs, method="ordinal")[0]) / 101
        viral_p_values.append(pvalue)
    ongoing_data[name + "_score"] = target_score
    ongoing_data[name + "_pvalue"] = viral_p_values


In [ ]:
# correlation of number of interactions
correlation_1 = round(sp.stats.pearsonr(ongoing_data["interactions_count"], ongoing_data["perm_3_days"])[0], 4)
fig = px.scatter(ongoing_data, x="interactions_count", y="perm_3_days", text="viral")
fig.update_traces(textposition="top center")
fig.update_layout(height=800, title_text=f"""num_interaction Vs Permeability, Pearson={correlation_1}""")
fig.show()

In [ ]:
# correlation of number of average score
correlation_2 = round(sp.stats.pearsonr(ongoing_data["network_score"], ongoing_data["perm_3_days"])[0], 4)
fig = px.scatter(ongoing_data, x="network_score", y="perm_3_days", text="viral")
fig.update_traces(textposition="top center")
fig.update_layout(height=800, title_text=f"""Network Score(average) Vs Permeability, Pearson={correlation_2}""")
fig.show()

In [25]:
# correlation of number of adj-pvalue random sources
correlation_4 = round(sp.stats.pearsonr(ongoing_data["adjusted_pvalue"], ongoing_data["perm_3_days"])[0], 4)
fig = px.scatter(ongoing_data, x="adjusted_pvalue", y="perm_3_days", text="viral")
fig.update_traces(textposition="top center")
fig.update_layout(height=800, title_text=f"""Adj(BH) P-value(random sources) Vs Permeability, Pearson={correlation_4}""")
fig.show()

In [ ]:
# correlation of number of adj-pvalue random networks
correlation_5 = round(sp.stats.pearsonr(ongoing_data["adjusted_pvalue_multi_net"], ongoing_data["perm_3_days"])[0], 4)
fig = px.scatter(ongoing_data, x="adjusted_pvalue_multi_net", y="perm_3_days", text="viral")
fig.update_traces(textposition="top center")
fig.update_layout(height=800, title_text=f"""Adjusted P-value(multi-net) Vs Permeability, Pearson={correlation_5}""")
fig.show()

In [27]:
# correlation of number of adj-pvalue random sources same degree
correlation_6 = round(sp.stats.pearsonr(ongoing_data["pvalue_random_source_same_degree"], ongoing_data["perm_3_days"])[0], 4,)
fig = px.scatter(ongoing_data, x="pvalue_random_source_same_degree", y="perm_3_days", text="viral")
fig.update_traces(textposition="top center")
fig.update_layout(height=800, title_text=f"""P-value(random source same degree) Vs Permeability, Pearson={correlation_6}""")
fig.show()